In [ ]:
import os
import numpy as np
import glob
import pickle
import json
import seaborn as sns
import matplotlib.ticker as ticker
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import plotly.graph_objects as go
#important for text to be detected when importing saved figures into illustrator
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
plt.rcParams["font.family"] = "Arial"

import utils

In [ ]:
"""
USER-DEFINED VARIABLES
"""

def define_params(method = 'single'):
    
    fparams = {}
    
    if method == 'single':
        
        fparams['fname_signal'] = 'VJ_OFCVTA_7_260_D6_neuropil_corrected_signals_15_50_beta_0.8.csv'   # 
        fparams['fname_events'] = 'framenumberforevents_VJ_OFCVTA_7_260_D6_trained.csv'
        fparams['fdir'] = os.path.abspath('./sample_data') # r'C:\Users\stuberadmin\Documents\GitHub\NAPE_imaging_postprocess\napeca_post\sample_data' # 
        fparams['fname'] = os.path.split(fparams['fdir'])[1]

        # set the sampling rate
        fparams['fs'] = 5

        # session info
        fparams['opto_blank_frame'] = False # if PMTs were blanked during stim, set stim times to nan (instead of 0)
        
        # analysis and plotting arguments
        fparams['num_rois'] = 10 # set to 'all' if want to show all cells
        fparams['conditions_2_analyze'] = ['minus', 'plus']
        fparams['flag_normalization'] = 'dff_perc' # options: 'dff', 'zscore', 'dff_perc', None
        
       
    elif method == 'f2a': # if string is empty, load predefined list of files in files_to_analyze_event

        fparams = files_to_analyze_event.define_fparams()

    elif method == 'root_dir':
        
        pass

    with open(os.path.join(fparams['fdir'], 'event_analysis_fparam'), 'w') as fp:
        json.dump(fparams, fp)
    
    return fparams

fparams = define_params(method = 'single') # options are 'single', 'f2a', 'root_dir'

if os.path.exists(os.path.join(fparams['fdir'], fparams['fname']+'.json')):
    with open(os.path.join(fparams['fdir'], fparams['fname']+'.json')) as json_file:
        json_data = json.load(json_file) 
        if 'fs' in json_data:
            fparams['fs'] = json_data['fs']


In [ ]:
cond_colors = ['steelblue', 'crimson', 'orchid', 'gold']

In [ ]:
# define load and save paths
fext = os.path.splitext(fparams['fname_signal'])[-1]
signals_fpath = os.path.join(fparams['fdir'], fparams['fname_signal'])

behav_file_path = os.path.join(fparams['fdir'], "framenumberforevents_*")

save_dir = os.path.join(fparams['fdir'], 'event_rel_analysis')

utils.check_exist_dir(save_dir); # make the save directory

In [ ]:
# functions to normalize traces
def calc_dff(activity_vec):
    mean_act = np.nanmean(activity_vec)
    return (activity_vec-mean_act)/mean_act

def calc_dff_percentile(activity_vec, perc=25):
    perc_activity = np.percentile(activity_vec, perc)
    return (activity_vec-perc_activity)/perc_activity

In [ ]:
# load time-series data
glob_signal_files = glob.glob(signals_fpath)
if len(glob_signal_files) == 1:
    if 'npy' in fext: 
        signals = np.squeeze(np.load(glob_signal_files[0]))
    elif 'csv' in fext:
        signals = pd.read_csv(glob_signal_files[0]).values
else:
    print('Warning: No or multiple signal files detected; using first detected file')
    
if fparams['opto_blank_frame']:
    try:
        glob_stim_files = glob.glob(os.path.join(fparams['fdir'], "{}*_stimmed_frames.pkl".format(fparams['fname'])))
        stim_frames = pickle.load( open( glob_stim_files[0], "rb" ) )
        signals[:,stim_frames['samples']] = None # blank out stimmed frames
        flag_stim = True
        print('Detected stim data; replaced stim samples with NaNs')
    except:
        flag_stim = False
        print('Note: No stim preprocessed meta data detected.')

if fparams['flag_normalization'] == 'dff':
    signal_to_plot = np.apply_along_axis(calc_dff, 1, signals)
elif fparams['flag_normalization'] == 'dff_perc':
    signal_to_plot = np.apply_along_axis(calc_dff_percentile, 1, signals)
else:
    signal_to_plot = signals

min_max = [list(min_max_tup) for min_max_tup in zip(np.min(signal_to_plot,axis=1), np.max(signal_to_plot,axis=1))]
min_max_all = [np.min(signal_to_plot), np.max(signal_to_plot)]

In [ ]:
if fparams['num_rois'] == 'all':
    fparams['num_rois'] = signals.shape[0]

total_session_time = signals.shape[1]/fparams['fs']
tvec = np.round(np.linspace(0, total_session_time, signals.shape[1]), 2)

In [ ]:
#load behavioral data and trial info
try:
    glob_frame_files = glob.glob(behav_file_path) # look for a file in specified directory
    if 'csv' in glob_frame_files[1]:
        event_frames = utils.df_to_dict(glob_frame_files[1])
    elif 'pkl' in glob_frame_files[1]:
        event_frames = pickle.load( open( glob_frame_files[1], "rb" ), fix_imports=True, encoding='latin1' ) # latin1 b/c original pickle made in python 2
except:
    print('Cannot find behavioral data file or file path is incorrect')

event_times = {}
if fparams['conditions_2_analyze']:
    for cond in fparams['conditions_2_analyze']:
        event_times[cond] = (np.array(event_frames[cond])/fparams['fs']).astype('int')


In [ ]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for idx_roi in np.arange(fparams['num_rois']):
    fig.add_trace(
        go.Scatter(
            visible=False,
            name='Activity',
            line=dict(color="green", width=1.5),
            x=tvec,
            y=signal_to_plot[idx_roi,:]))


# add vertical lines for events 
for idx_cond, cond in enumerate(fparams['conditions_2_analyze']):
    for idx_ev, event in enumerate(event_times[cond]):
        # 2nd case used to hide trace duplicates in legend
        if idx_ev==0:
            fig.add_trace(go.Scatter(x=[event,event],y=[min_max_all[0],min_max_all[1]], visible=True,  
                                     mode='lines', 
                                     line=dict(color=cond_colors[idx_cond], width=1.5, dash='dash'), showlegend=True, 
                                     legendgroup=cond,
                                     name='{}'.format(cond)))
        else:
            fig.add_trace(go.Scatter(x=[event,event], y=[min_max_all[0],min_max_all[1]], visible=True,
                                     mode='lines',  
                                     showlegend=False, legendgroup=cond,
                                     line=dict(color=cond_colors[idx_cond], width=1.5, dash='dash'),
                                     name='{} {}'.format(cond, str(idx_ev))))
    

# make a list of attributes for slider steps
steps = []
for iroi in np.arange(fparams['num_rois']): 
    # when ever slider changes, set all roi's to be invisible and all event lines as visible
    step = dict(
        method="restyle",
        # layout attributes
        args=[{"visible": ([False] * fparams['num_rois']) + [True] * (len(fig.data)- fparams['num_rois'])},
              {"title": "Viewing ROI " + str(iroi)},
              ])
    # then make the selected ROI visible
    step["args"][0]["visible"][iroi] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# create and setup slider 
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Viewing "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="Fluorescence ({})".format(fparams['flag_normalization']),
    legend_title="Legend Title",
    font=dict(
        size=12),
    sliders=sliders,
    showlegend=True,
    legend_title_text='Legend',
    plot_bgcolor='rgba(0,0,0,0)'
)

# rename slider ticks
for idx in np.arange(fparams['num_rois']):
    fig['layout']['sliders'][0]['steps'][idx]['label']='ROI ' + str(idx)

# Make 1st trace visible
fig.data[0].visible = True

# Change grid color and axis colors
fig.update_xaxes(showline=True, linewidth=1.5, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1.5, linecolor='black')

fig.show()

In [ ]:
# this only outputs the un-altered original plot in vectorized format
fig.write_image(os.path.join(save_dir, 'whole_session_event.pdf'))